# Data-Set Secondary Pre-Processing

This notebook further focusses on processing steps we can use to enhance the images in our data-sets

## Author: Alexander Goudemond, Student Number: 219030365

# Imports

# Previous Results:

This section of the notebook summarises the notes from 004

1105, 1106, 1201 and 1202 are useful to show the user the hidden information in the images. 

0801 and 0802 are super close at being useful, however, they posses undesirable features

0805, 1101 and 1103 may be useful for segmentation

It may be worth attempting to try introduce an AND gate between 2 images, in order to extract only the common information. Though this may preserve noise... 

To get an idea of what we are hoping to achieve, let us open 2 images from one of our reference materials: Magnussen (PhD Thesis)

The images will be available in the COMP700 Proposal, if not provided by the author



Segmentation Example


![Segmentation Example](Magnusson_Segmentation_Sample.png "Segmentation Example")

Tracking Example Below:

As we can see above, in the segmentation example from Magnussen, (b) is a desirable Processing step before segmentation

At the moment, we have the following desirable affects:


Hidden Info present (1105, 1106, 1201, 1202):


![Hidden Info 1](004_InitialPreProcessing\1105_ErodedNegativeImages.png "Hidden Info 1")

![Hidden Info 2](004_InitialPreProcessing\1106_OpenedNegativeImages.png "Hidden Info 2")

![Hidden Info 3](004_InitialPreProcessing\1201_HistEq_ErodedAtEnd_BlurAdaptiveThresholdMean.png "Hidden Info 3")

![Hidden Info 4](004_InitialPreProcessing\1202_HistE1_OpenedAtStart_BlurAdaptiveThresholdMean.png "Hidden Info 4")

Almost pleasant processed Images (0801, 0802):

![Almost Pleasant Processed Images 1](004_InitialPreProcessing\0801_OtsuThreshold.png "Almost Pleasant Processed Images 1")

![Almost Pleasant Processed Images 2](004_InitialPreProcessing\0802_Blur_OtsuThreshold.png "Almost Pleasant Processed Images 2")


Potential Useful Images (0805, 1101, 1103):


![Potentially Useful Images 1](004_InitialPreProcessing\0805_BlurAdaptiveThresholdMean.png "Potentially Useful Images 1")

![Potentially Useful Images 2](004_InitialPreProcessing\1101_ErodedImages.png "Potentially Useful Images 2")

![Potentially Useful Images 3](004_InitialPreProcessing\1103_OpenedImages.png "Potentially Useful Images 3")

As we can see from the 3 pictures above, The processing we are attempting to produce is not of the same quality of Magnussen YET

One of the challenges we are facing is to find a processing step that assists all 10 data-sets, because if we only focussed on 1 data-set we would be able to pick a specific processing option... 

For ease of reference, bottom left of the images above corresponds to Magnussen:

![Segmentation Example](Magnusson_Segmentation_Sample.png "Segmentation Example")

From the above images, we can visualy see that 1103 is the closest to improving things, but it is rather dark. We need to try use this technique to brighten the overall image, so that we can move forward. The other techniques tried in 004 may be unhelpful as they either help a few data-sets and damage others, or introduce noise...

We will carefully explore the combinations here, though

Infact, upon closer inspection, 1105 and 1106 may be useful after all... Let us carefully compare and contrast 1101, 1103, 1105, 1106

![1101](004_InitialPreProcessing\1101_ErodedImages.png "1101")

![1103](004_InitialPreProcessing\1103_OpenedImages.png "1103")

![1105](004_InitialPreProcessing\1105_ErodedNegativeImages.png "1105")

![1106](004_InitialPreProcessing\1106_OpenedNegativeImages.png "1106")

Between 1101 and 1103, 1103 is better!

Between 1105 and 1106, 1106 appears better!

Let ys carefully consider 1103 vs 1106

![1103](004_InitialPreProcessing\1103_OpenedImages.png "1103")

![1106](004_InitialPreProcessing\1106_OpenedNegativeImages.png "1106")

Both 1103 and 1106 contain useful information! We should create videos from them, to cover whether the results are universally good. We are also interested in whether these formatting tools assist with the test cases

Of course they are both good! One is an opened image, the other is the negative of it!

So instead, let us create a video using the technique from 1106 as well as a video from 0805 to see if our results are helpful

#  1106 and 0805 Videos

This section of the notebook focusses on videos using the techniques present in 0805 and 1106, identified in the notebook 004

This section will closely follow the system developed in 003

We need a variable containing the Directory Names

We need to generate a collection of locations from each of those Directories, for each data-set

We need a variable containing the Image Sizes

We need 3 functions: processImage, generateVideo, moveVideo

Thereafter, we can create small sample videos and save them here, for ease of reference

Directories:

In [1]:
from os import getcwd, walk

data_sets = "..\\..\\Comp700_DataSets"
current_directory = getcwd()

path = walk(current_directory + "\\" + data_sets)

directory_array = [] # contains the main folders

i = 1
for root, dirs, files in path:
    if (i == 2):
        directory_array = dirs
        break
    
    i += 1

print("Directory Array")
print(directory_array)

Directory Array
['BF-C2DL-HSC', 'BF-C2DL-HSC (1)', 'BF-C2DL-MuSC', 'BF-C2DL-MuSC (1)', 'DIC-C2DH-HeLa', 'DIC-C2DH-HeLa (1)', 'Fluo-C2DL-Huh7', 'Fluo-C2DL-Huh7 (1)', 'Fluo-C2DL-MSC', 'Fluo-C2DL-MSC (1)', 'Fluo-N2DH-GOWT1', 'Fluo-N2DH-GOWT1 (1)', 'Fluo-N2DH-SIM+', 'Fluo-N2DH-SIM+ (1)', 'Fluo-N2DL-HeLa', 'Fluo-N2DL-HeLa (1)', 'PhC-C2DH-U373', 'PhC-C2DH-U373 (1)', 'PhC-C2DL-PSC', 'PhC-C2DL-PSC (1)']


Locations:

In [2]:
# First, generate a list of the locations for each folder of Petri Dish images

path = walk(current_directory + "\\" + data_sets) # reset path

location_array = []

# used to cycle between 2 folders, present in each directory
sub_directory_choice = ["\\01\\", "\\02\\"]

i = 0 # will grow from 0 to 39
index = 0 # we need to lie in [0, 19]
for root, dirs, files in path:
    # print(dirs)
    for item in files:
        if ("t0000.tif" == item) or ("t000.tif" == item):
            index = i // 2

            location = ( current_directory + "\\" + data_sets + "\\Extracted\\" + directory_array[index] + 
                        "\\" + directory_array[index] + sub_directory_choice[i % 2])
            
            i += 1
            # print(location)
            location_array.append(location)

print(location_array)



['c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-HSC\\BF-C2DL-HSC\\01\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-HSC\\BF-C2DL-HSC\\02\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-HSC (1)\\BF-C2DL-HSC (1)\\01\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-HSC (1)\\BF-C2DL-HSC (1)\\02\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-MuSC\\BF-C2DL-MuSC\\01\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-MuSC\\BF-C2DL-MuSC\\02\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-MuSC (1)\\BF-C2DL-MuSC (1)\\01\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\\..\\Comp700_DataSets\\Extracted\\BF-C2DL-MuSC (1)\\BF-C2DL-MuSC (1)\\02\\', 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\..\

Image Sizes:

We only need to scan the dimensions of the first image as all images in a folder contain the same dimensions!

In [3]:
import cv2

image_size_array = []

path = walk(current_directory + "\\" + data_sets) # reset path

i = -1 # will grow from 0 to 39
for root, dirs, files in path:
    for item in files:
        if ("man_" not in item) and (".zip" not in item):
            # update on first element only
            if ("t0000.tif" == item) or ("t000.tif" == item):  
                i += 1

            img = cv2.imread( (location_array[i] + item), cv2.IMREAD_GRAYSCALE)
            (x, y) = img.shape

            image_size_array.append([x, y])
            break
            

        # skip "man_" images
        else:
            break

print(image_size_array)


[[1010, 1010], [1010, 1010], [1010, 1010], [1010, 1010], [1036, 1070], [1036, 1070], [1036, 1070], [1036, 1070], [512, 512], [512, 512], [512, 512], [512, 512], [1024, 1024], [1024, 1024], [1024, 1024], [1024, 1024], [832, 992], [782, 1200], [832, 992], [782, 1200], [1024, 1024], [1024, 1024], [1024, 1024], [1024, 1024], [690, 628], [773, 739], [718, 660], [790, 664], [700, 1100], [700, 1100], [700, 1100], [700, 1100], [520, 696], [520, 696], [520, 696], [520, 696], [576, 720], [576, 720], [576, 720], [576, 720]]


In [4]:
from os import remove, mkdir
from os.path import exists, join
import cv2
import numpy as np
import matplotlib.pyplot as plt

kernel = np.ones((3,3), np.uint8)

# create directory for work we create
def tryMakeDirectory(current_directory, destination_directory):
    try:
        # join comes from os.path
        mkdir( join(current_directory, destination_directory) )
    except FileExistsError:
        print("Folder already exists!")
        pass
    except:
        print("Unknown Error Encountered...")
###

def openImage(image):
    img_erode = cv2.erode(image, kernel, iterations=1)
    img_opened = cv2.dilate(img_erode, kernel, iterations=1)

    max_val = np.amax(img_opened)
    neg_img = max_val - img_opened

    return neg_img
###

def blurredAdaptiveThreshold(image):
    max_value = np.amax(image)

    blur = cv2.GaussianBlur(image, (5, 5), 0)
    thresh_img = cv2.adaptiveThreshold(blur, max_value, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, blockSize=11, C=2)

    return thresh_img
###

# processingChoice == 1 or 2
def generateVideos(current_directory, desired_folder, processingChoice):
    # only progress if files don't exist
    makeVideos = False

    if (exists(current_directory + "\\" + desired_folder)):
        # Now, go to directory and verify all is there
        path = walk(current_directory + "\\" + desired_folder)

        count = 0
        for root, dirs, files in path:
            for item in files:
                count += 1

        if (count == 40):
            print("All Videos exist already!")
        else:
            print("Not all Videos exist")
            makeVideos = True
    else:
        makeVideos = True
        
    if (makeVideos):
        path = walk(current_directory + "\\" + data_sets) # reset path

        sub_directory_choice = ["01", "02"]

        i = -1
        output_video = cv2.VideoWriter()
        frames_per_second = 10
        petri_dish_images = False

        # Generates Colour Videos
        for root, dirs, files in path:
            for item in files:
                if ("man_" not in item) and (".zip" not in item):
                    # update on first element only
                    if ("t0000.tif" == item) or ("t000.tif" == item):  

                        if (processingChoice == 1):
                            fileName = "blurredAdaptiveThreshold"
                        else:
                            fileName = "Opened"

                        petri_dish_images = True
                        i += 1
                        index = i // 2 # used for output video as 2 copies for each directory

                        size = (image_size_array[i][1], image_size_array[i][0] ) # notice order
                        fileName += "_" + directory_array[index] + "_" + sub_directory_choice[i % 2] + ".mp4"
                        
                        output_video = cv2.VideoWriter(
                            fileName, 
                            cv2.VideoWriter_fourcc(*'DIVX'), 
                            frames_per_second, 
                            size,
                            isColor=False # either True or False
                        )

                    img = plt.imread( location_array[i] + item) 
                    plt.imsave("temp.jpg", img, cmap="gray") # desired colourmap for us
                    img = cv2.imread( "temp.jpg", cv2.IMREAD_GRAYSCALE)

                    if (processingChoice == 1):
                        new_img = blurredAdaptiveThreshold(img)
                    else:
                        new_img = openImage(img)
                        
                    output_video.write(new_img)

                else:
                    petri_dish_images = False
                    break
                    

            if (petri_dish_images):     
                cv2.destroyAllWindows()
                output_video.release()
                print("Video finished for ", fileName, sep="")
                petri_dish_images = False # update incase next iteration containes empty array
    
    # remove at end
    if (exists("temp.jpg")):
        remove("temp.jpg")

In [5]:
from shutil import move # moves and replaces files

def moveBulkVideos(current_directory, desired_folder):
    # only progress if files don't exist
    if (exists(current_directory + "\\" + desired_folder)):
        print("Videos already exist!")
    else:
        # local function
        tryMakeDirectory(current_directory, desired_folder)

        path = walk(current_directory)

        for root, dirs, files in path:
            for item in files:
                if (".mp4" in item):
                    new_destination = current_directory + "\\" + desired_folder
                    move(join(current_directory, item), join(new_destination, item)) # should overwrite existing data

        # Now, go to directory and verify all is there
        path = walk(current_directory + "\\" + desired_folder)

        count = 0
        for root, dirs, files in path:
            for item in files:
                count += 1

        if (count == 40):
            print("All Videos Moved Successfully!")
        else:
            print("Not all Videos Moves Successfully")

In [6]:
generateVideos(current_directory, "..\\..\\COMP700_VideosOfDataSets_BlurAdaptiveThreshold", 1)

Video finished for blurredAdaptiveThreshold_BF-C2DL-HSC_01.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-HSC_02.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-HSC (1)_01.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-HSC (1)_02.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-MuSC_01.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-MuSC_02.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-MuSC (1)_01.mp4
Video finished for blurredAdaptiveThreshold_BF-C2DL-MuSC (1)_02.mp4
Video finished for blurredAdaptiveThreshold_DIC-C2DH-HeLa_01.mp4
Video finished for blurredAdaptiveThreshold_DIC-C2DH-HeLa_02.mp4
Video finished for blurredAdaptiveThreshold_DIC-C2DH-HeLa (1)_01.mp4
Video finished for blurredAdaptiveThreshold_DIC-C2DH-HeLa (1)_02.mp4
Video finished for blurredAdaptiveThreshold_Fluo-C2DL-Huh7_01.mp4
Video finished for blurredAdaptiveThreshold_Fluo-C2DL-Huh7_02.mp4
Video finished for blurredAdaptiveThreshold_Fluo-C2DL-Huh7 (1)_01.mp4
Video 

In [8]:
moveBulkVideos(current_directory, "..\\..\\COMP700_VideosOfDataSets_BlurAdaptiveThreshold")

Videos already exist!


Those videos were generated successfully, however they have an incredible amount of noise and further processing may disrupt the cell borders. Let us next focus on generating the Opened Videos:

In [9]:
generateVideos(current_directory, "..\\..\\COMP700_VideosOfDataSets_Opened", 2)

Video finished for Opened_BF-C2DL-HSC_01.mp4
Video finished for Opened_BF-C2DL-HSC_02.mp4
Video finished for Opened_BF-C2DL-HSC (1)_01.mp4
Video finished for Opened_BF-C2DL-HSC (1)_02.mp4
Video finished for Opened_BF-C2DL-MuSC_01.mp4
Video finished for Opened_BF-C2DL-MuSC_02.mp4
Video finished for Opened_BF-C2DL-MuSC (1)_01.mp4
Video finished for Opened_BF-C2DL-MuSC (1)_02.mp4
Video finished for Opened_DIC-C2DH-HeLa_01.mp4
Video finished for Opened_DIC-C2DH-HeLa_02.mp4
Video finished for Opened_DIC-C2DH-HeLa (1)_01.mp4
Video finished for Opened_DIC-C2DH-HeLa (1)_02.mp4
Video finished for Opened_Fluo-C2DL-Huh7_01.mp4
Video finished for Opened_Fluo-C2DL-Huh7_02.mp4
Video finished for Opened_Fluo-C2DL-Huh7 (1)_01.mp4
Video finished for Opened_Fluo-C2DL-Huh7 (1)_02.mp4
Video finished for Opened_Fluo-C2DL-MSC_01.mp4
Video finished for Opened_Fluo-C2DL-MSC_02.mp4
Video finished for Opened_Fluo-C2DL-MSC (1)_01.mp4
Video finished for Opened_Fluo-C2DL-MSC (1)_02.mp4
Video finished for Opened_Fl

In [10]:
moveBulkVideos(current_directory, "..\\..\\COMP700_VideosOfDataSets_Opened")

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\G5\\Documents\\GitHub\\COMP700\\BF-C2DL-HSC_01.mp4'